In [ ]:
# imports
import numpy as np
import rasterio
import glob
import os

from scipy.stats import linregress
from numpy.polynomial.polynomial import polyval
from numpy import polyfit

In [ ]:
# all input folders for NPP, NEE and SR trends statistics (were loaded one at a time because of the same name of the following variable)

#input_folder_nee = 'E:/diplom/GCAS/gcas_bio_opt_2010_2019'
#output_nee_r2 = 'E:/diplom/py_trends/nee_10_19_r2_polynom.tif'

#input_folder_npp = 'E:/diplom/py_trends/npp_resample'
#output_npp_r2 = 'E:/diplom/py_trends/npp_10_19_r2.tif'

input_folder_sr = 'E:/diplom/py_trends/sr_resample'
output_sr_r2 = 'E:/diplom/py_trends/sr_10_19_r2.tif'

In [ ]:
tif_files = sorted(glob.glob(os.path.join(input_folder_nee, 'gcas_bio_opt_20*.tif')))
years = list(range(2010, 2020))

In [ ]:
tif_files = sorted(glob.glob(os.path.join(input_folder_npp, 'npp201*_steppe_kgC_5km.tif')))
years = list(range(2010, 2020))

In [38]:
tif_files = sorted(glob.glob(os.path.join(input_folder_sr, 'SR*.tif')))
years = list(range(2010, 2020))

In [39]:
arrays = []
for f in tif_files:
    with rasterio.open(f) as src:
        arrays.append(src.read(1))
        if 'profile' not in locals():
            profile = src.profile

In [ ]:
# data arrays to the same format 
min_rows = min(arr.shape[0] for arr in arrays)
min_cols = min(arr.shape[1] for arr in arrays)
arrays_clipped = [arr[:min_rows, :min_cols] for arr in arrays]
data = np.stack(arrays_clipped) 

r2 = np.full(data.shape[1:], np.nan, dtype=np.float32) # filling in a new NaN array for writing R2 later

In [ ]:
# linear regeression
for i in range(data.shape[1]):
    for j in range(data.shape[2]):
        y = data[:, i, j]
        if np.all(np.isnan(y)) or np.sum(~np.isnan(y)) < 5:
            continue
        if np.any(np.abs(y) > 1e6):
            continue
        _, _, r_value, _, _ = linregress(years, y) # the slope was calculated in QGIS earlier, we need the statistics only
        r2[i, j] = r_value ** 2

In [ ]:
# polynom 2deg regression
for i in range(data.shape[1]):
    for j in range(data.shape[2]):
        y = data[:, i, j]
        if np.all(np.isnan(y)) or np.sum(~np.isnan(y)) < 5:
            continue
        if np.any(np.abs(y) > 1e6):
            continue

        x = np.array(years)

        coeffs = np.polyfit(x, y, deg=2)
        y_pred = np.polyval(coeffs, x)
        rss = np.sum((y - y_pred) ** 2) # calculating Residual Sum of Squares, RSS
        tss = np.sum((y - np.nanmean(y)) ** 2) # calculating Total Sum of Squares, TSS
        if tss < 1e-6:
            continue
        r2[i, j] = 1 - rss / tss

        # to get average polynom trend:
        #trend_vals = 2 * coeffs[0] * x + coeffs[1]
        #avg_trend = np.mean(trend_vals)
        #slope[i, j] = avg_trend 

In [ ]:
# export to raster format
profile.update(dtype=rasterio.float32, count=1)
with rasterio.open(output_sr_r2, 'w', **profile) as dst:
    dst.write(r2, 1)